In [3]:
import pandas as pd
import os
import uol_redacoes_xml
import numpy as np

In [4]:
# %pip install scikit-learn
# %pip install matplotlib
# pip install git+https://github.com/gpassero/uol-redacoes-xml.git
# %pip install scrapy
# %pip install pyquery
# %pip install html2text
# %pip install bs4
# %pip install scikit-learn
# %pip install nltk
# %pip install spacy

In [5]:
%reload_ext watermark
%watermark -a "Rodrigo W Pisaia" --iversion --python
!nvidia-smi

Author: Rodrigo W Pisaia

Python implementation: CPython
Python version       : 3.10.16
IPython version      : 8.30.0

numpy           : 2.2.4
uol_redacoes_xml: 0.2
pandas          : 2.2.3

Sun Apr  6 12:11:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.83                 Driver Version: 572.83         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060      WDDM  |   00000000:07:00.0  On |                  N/A |
| 40%   32C    P0             40W /  170W 

In [6]:
essays = uol_redacoes_xml.load()
print([attr for attr in essays[0].__dir__() if not attr.startswith('_')])

['title', 'text', 'criteria_scores', 'final_score', 'prompt', 'url', 'fixed_text', 'errors', 'comments', 'get_features']


In [ ]:
# create dataframe from essays
df_essays = pd.DataFrame([vars(essay) for essay in essays])
print(f"\ndf.shape:{df_essays.shape}")
print(f"\ndf.dtypes:\n{df_essays.dtypes}")
df_essays.head(1)


df.shape:(2164, 9)

df.dtypes:
title               object
text                object
criteria_scores     object
final_score        float64
prompt              object
url                 object
fixed_text          object
errors              object
comments            object
dtype: object


,title,text,criteria_scores,final_score,prompt,url,fixed_text,errors,comments
0,Conflito entre liberdade de expressão e intimi...,"A liberdade de expressão, assim como o direito...","{'Competência 1': 1.0, 'Competência 2': 1.0, '...",5.0,Direitos em conflito: liberdade de expressão e...,https://educacao.uol.com.br/bancoderedacoes/re...,"A liberdade de expressão, assim como o direito...","[(intimidade, intimidade,), (contato, ), (auto...",<comments>## Comentário geral\nTexto regular. ...


In [ ]:
scores_df = pd.json_normalize(df_essays['criteria_scores'])
scores_df = scores_df.add_prefix('score_')
df_essays = pd.concat([df_essays.drop('criteria_scores', axis=1), scores_df], axis=1)

In [ ]:

df_essays.columns = [
    "title",
    "text",
    "final_score",
    "prompt",
    "url",
    "fixed_text",
    "errors",
    "comments",
    "score_comp_1",
    "score_comp_2",
    "score_comp_3",
    "score_comp_4",
    "score_comp_5",
]
df_essays.head(2)

,title,text,final_score,prompt,url,fixed_text,errors,comments,score_comp_1,score_comp_2,score_comp_3,score_comp_4,score_comp_5
0,Conflito entre liberdade de expressão e intimi...,"A liberdade de expressão, assim como o direito...",5.0,Direitos em conflito: liberdade de expressão e...,https://educacao.uol.com.br/bancoderedacoes/re...,"A liberdade de expressão, assim como o direito...","[(intimidade, intimidade,), (contato, ), (auto...",<comments>## Comentário geral\nTexto regular. ...,1.0,1.0,1.0,1.0,1.0
1,Entre direitos individuais e escândalos políticos,Os direitos fundamentais contidos na Constitui...,6.5,Direitos em conflito: liberdade de expressão e...,https://educacao.uol.com.br/bancoderedacoes/re...,Os direitos fundamentais contidos na Constitui...,"[(quais, que), (análise, ), (há, a), (não., nã...","<comments>## Comentário geral\nTexto regular, ...",1.0,2.0,1.5,1.0,1.0


In [ ]:
numeric_cols = df_essays.select_dtypes(np.number).columns
df_essays2 = df_essays.copy()
for col in numeric_cols:
    df_essays2[col] = df_essays2[col] * 100

df_essays2["errors"] = df_essays2["errors"].astype(str)
df_essays2["title"] = df_essays2["title"].astype(str)
df_essays2["prompt"] = df_essays2["prompt"].astype(str)

df_essays2.head(1)

,title,text,final_score,prompt,url,fixed_text,errors,comments,score_comp_1,score_comp_2,score_comp_3,score_comp_4,score_comp_5
0,Conflito entre liberdade de expressão e intimi...,"A liberdade de expressão, assim como o direito...",500.0,Direitos em conflito: liberdade de expressão e...,https://educacao.uol.com.br/bancoderedacoes/re...,"A liberdade de expressão, assim como o direito...","[('intimidade', 'intimidade,'), ('contato', ''...",<comments>## Comentário geral\nTexto regular. ...,100.0,100.0,100.0,100.0,100.0


In [ ]:
df_essays3 = df_essays2.copy().drop_duplicates().reset_index(drop=True)
date_today = pd.to_datetime("today").strftime("%Y_%m_%d")
df_essays3.to_parquet(f"../02_output/{date_today}_essays.parquet", index=False)